- Sorting is quite an integral part of many algorithms, because a sorted input can speed up many algorithms, and can also speed up our queries

- In this section, we study the efficient ways to sort something

- Sorting problem
    - **Input:** Sequence $A[1...N]$
    - **Output:** Permutation $A'[1...N]$ in increasing/decreasing order

### Selection Sort

- We start with the naive solution, called **selection sort**

- Basically, for each index in the $n$ length input array, we scan through the entire input to place the minimum at each step. 
    - At step 1, we make $n-1$ comparisons
    - At step 2, we make $n-2$ comparisons (because the first element is already minimum)
    - etc

- In total, we make $(n-1) + (n-2) + ...$ comparisons, or $\frac{n * (n+1)}{2}$ comparisons. This gives us a run time of $O(N^2)$

In [3]:
def selection_sort(array: list):
    for i in range(len(array)):
        min_index = i
        for j in range(i, len(array)):
            if array[j] < array[min_index]:
                min_index = j
        array[i], array[min_index] = array[min_index], array[i]
    return array

### Merge Sort

- A faster way to sort an array (and also what we've covered in the previous section) is to make use of divide and conquer

- The idea behind merge sort is to split the array iteratively into smaller and smaller segments, then put them back together in sorted order. Intuitively, at each recursive step, we are comparing arrays that are already sorted, so you don't need to make $n^2$ comparisons. Instead, you just compare your solution with the relevant value

- Merge sort satisfies the recursion $T(N) = 2 \cdot T(\frac{N}{2}) + O(N)$, where $a = 2, b=2, d=1$
    - $a=2$ because at each step, we're recursively splitting the array into 2 parts, and calling merge sort on each part
    - $b=2$ because the array size is halved at each step
    - $d=1$ because the merge step requires $N$ comparisons at the worst case

- By Master Theorem, since $\log_b(a) = \log_2(2) = 1 = d$, run time is therefore $O(N \log(N))$

In [30]:
def merge(left_array: list, right_array: list, verbose: bool):
    '''
    Time complexity: O(N), because in the worst case, you need to make $N-1$ comparisons
    Space compexity: O(N), because you're creating a new ordered list, no other intermediates
    '''

    merged_array = []
    while (len(left_array) != 0) & (len(right_array) != 0):
        if verbose:
            print(f"{merged_array=}")
        if left_array[0] <= right_array[0]:
            val = left_array.pop(0)
        else:
            val = right_array.pop(0)
        
        if verbose:
            print(f'{val=}')
        merged_array.append(val)
    
    if verbose:
        print(f"{merged_array=}")
    
    if len(left_array) > 0:
        merged_array += left_array
    elif len(right_array) > 0:
        merged_array += right_array
    
    return merged_array

def merge_sort(array: list, verbose: bool):
    '''
    Time complexity: O(N log(N)). O(N) for the merge step, and O(log(N)) for the recursive step
    Space compexity: O(N), because you're creating a new ordered list, no other intermediates
    '''

    if verbose:
        print('='*50)

    if len(array) == 1:
        return array
    
    mid_index = len(array)//2
    left_array = merge_sort(array[:mid_index], verbose=verbose)
    right_array = merge_sort(array[mid_index:], verbose=verbose)
    
    if verbose:
        print(f'{mid_index=}, {left_array=}, {right_array=}')
    
    merged_array = merge(left_array, right_array, verbose)
    
    if verbose:
        print(f"{merged_array=}, {left_array=}, {right_array=}")
    
    return merged_array


merge_sort([1,1,522,6,14,71,51,4452,7812,3451,34,1352,7,4,141,36,3], False)

[1, 1, 3, 4, 6, 7, 14, 34, 36, 51, 71, 141, 522, 1352, 3451, 4452, 7812]

### Comparison-based vs Non-comparison based sorting

- Comparison-based sorting is when sorting is done via some comparison of 2 elements. The 2 sorting approaches we have studied so far (merge sort and selection sort) are both comparison based sorts

- In general, for comparison based sorting, the algorithm performs $\Omega(n \log n)$ comparisons in the worst case to sort $n$ objects
    - That is, for any comparison based sorting algorithm, there exists an array $A$ such that the algorithm performs at least $\Omega(n \log n)$ comparisons to sort $A$

- Proof
    - For a given array, the sequence of comparisons made can be expressed in terms of a decision tree. 
        - Imagine an array [3,2,1]. To sort it, we must first compare 3 vs 2, then 3 vs 1, then 2 vs 1 
        - Every leaf in this tree is a specific sequence of comparisons
    - In general, the number of leaves $L$ for a given array of size $N$ must be at least $N!$
    - Why? Because an array of size $N$ has $N!$ possible permutations. And so there must be $N!$ ways to make the comparisons such that you get to the sorted solution. 
    - The worst case run time is given by the depth $d$ of the tree
    - However, the depth of the tree is directly related to the number of leaves too!
        - It must be the case that $2^d \ge L$, assuming a binary tree
        - Rewriting, $d \log_2(2) \ge \log_2(L) \rightarrow d \ge \log_2(L)$
    - Since depth $d$ is worst case run time, it must be that the worst case run time is $\log_2(L) = \log_2(N!)$
    $$\begin{aligned}
        \log_2(N!) &= \log_2(1 \cdot 2 \cdot ... N) \\
        &= \log_2(1) + \log_2(2) + ... \log_2(N) \\
        &\ge \log_2(\frac{N}{2}) + \log_2(\frac{N}{2} + 1) + ... \log_2(N) & \text{throwing away the first half of the summation leads to a smaller number} \\
        &\ge \frac{n}{2} \log_2(\frac{N}{2}) & \text{there are n/2 values in both sums, but every term in this sum is lower than the previous}\\
        &= \Omega(n \log n) & \text{Dropping constants}
    \end{aligned}$$ 


- However, comparing pairs of value is not the only way to sort an array, which is why we have **non comparison-based sorting**

- Here, we look at an example called **Count sort**, which is kind of a gimmicky sort that works only when the values in the array are small integers.

- Let's look at how this works:
    - Imagine my input array is [1,2,5,1,2,3,1,5,1,4,2]
    - To sort this array, we iterate through the input array and count the number of values 1, 2, 3, 4, 5
    - We conclude that there are four 1s, three 2s, one 3, one 4, and one 5
    - So our sorted array is just [1,1,1,1,2,2,2,3,4,5]. This was sorted without explicitly comparing anything, and we only scan the array once! 

In [53]:
import numpy as np

def count_sort(array, max_val=100):
    '''
    Time complexity: O(2 * array_size + max_val) = O(N + M)
    Space complexity: O(array_size + max_val) for storing list and generating sorted list
    '''
    # print(array)
    assert max(array) < max_val
    array_len = len(array)


    # This portion is O(N)
    counts = [0] * max_val
    for i in range(array_len):
        counts[array[i]] += 1
    # print(counts)

    # This portion is O(M)
    start_positions = [0]*max_val
    start_positions[0] = 0
    for j in range(1, max_val):
        start_positions[j] = start_positions[j-1] + counts[j-1]
    # print(start_positions)

    # This portion is O(N)
    sorted_array = [0]*array_len
    for k in range(array_len):
        element_to_insert = array[k]
        insert_at_index = start_positions[element_to_insert]
        # print(f"{k=}, {element_to_insert=}, {insert_at_index=}")
        
        sorted_array[insert_at_index] = element_to_insert
        start_positions[element_to_insert] += 1
    
    return sorted_array

array = np.random.randint(0,99,size=3)
count_sort(array)

[41, 78, 93]